In [175]:
import glob
import os
import pandas as pd
import numpy as np

In [176]:
from tkinter import *
 
window = Tk()
 
window.title("PWG precision")
 
window.geometry('750x200')
 
lbl = Label(window, text="input path (day1)")
lbl_2 = Label(window, text="input path (day2)")
lbl_3 = Label(window, text="input path (day3)")
lb2 = Label(window, text="saving path")
lb3 = Label(window, text="Full sites only?")
lb4 = Label(window, text="3-day or 1-day precision?")
lb5 = Label(window, text="")
 
lbl.grid(column=0, row=0)
lbl_2.grid(column=0, row=1)
lbl_3.grid(column=0, row=2)
lb2.grid(column=0, row=3)
lb3.grid(column=0, row =4)
lb4.grid(column=0, row =5)
lb5.grid(column=0, row =6)
 
input_p = Entry(window,width=100)
input_p_2 = Entry(window,width=100)
input_p_3 = Entry(window,width=100)
output_p = Entry(window, width=100)

var = StringVar(window)
var.set('yes')
full_s = OptionMenu(window, var, 'yes','no')
full_s.place(relx=0,x=5, y=5)

var1 = StringVar(window)
var1.set('1-day')
nday = OptionMenu(window, var1, '1-day','3-day')
nday.place(relx=0,x=5, y=5)
 
input_p.grid(column=1, row=0)
input_p_2.grid(column=1, row=1)
input_p_3.grid(column=1, row=2)
output_p.grid(column=1, row=3)
full_s.grid(column=1,row=4)
nday.grid(column=1,row=5)

 
def clicked(): 
    global input_file, input_file2, input_file3, output_path, fullsite_only, three_day
    input_file = input_p.get()
    input_file2 = input_p_2.get()
    input_file3 = input_p_3.get()
    output_path = output_p.get()
    
    temp = var.get()
    if temp =='yes': fullsite_only=True
    else: fullsite_only=False
    
    temp1 = var1.get()
    if temp1 == '1-day': three_day = False
    else: three_day = True
    
    window.destroy()
    
btn = Button(window, text="Submit", command=clicked)
 
btn.grid(column=1, row=8)
 
window.mainloop()

In [ ]:
#fullsite_only = False
#three_day = True
#files_path = r'C:\Users\xshen\Documents\Python Scripts\sample_data\*.csv'
#output_path = r'C:\Users\xshen\Documents\Python Scripts\sample_data\temp\combined\precision.csv'

#3-day inputs
#input_file1 = r'C:\Users\xshen\Documents\Python Scripts\sample_data\temp\day1\*.csv'
#input_file2 = r'C:\Users\xshen\Documents\Python Scripts\sample_data\temp\day2\*.csv'
#input_file3 = r'C:\Users\xshen\Documents\Python Scripts\sample_data\temp\day3\*.csv'

In [177]:
def header_info(file_path):
    """
    Extract measurement info from header
    
    Arg:
    ====
    file_path: directory where input csv is saved, string
    
    Returns"
    ====
    header_ind: num of rows that contain header info, int
    res: resolution used for the measurement, string
    EE: edge exclusion used for the measurement, float
    """
    
    with open(file_path) as fp:
        for ind, line in enumerate(fp):
            if line.startswith('Edge Exclusion:'):
                EE = line.split(',')[1]
                EE = EE.strip('\n')
            if line.startswith('Spatial Resolution:'):
                res = line.split(',')[1]
                res = res.strip('\n')
            if line.startswith('=========='): 
                header_ind = ind+1
                break
    return header_ind, res, EE

In [178]:
def M49(df):
    """
    compute precision for M49 file
    
    Arg:
    ====
    df: M49 data, dataframe
    
    Return:
    ====
    list of standard deviation for 4 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)

    GBIR = np.power(df.groupby(by=['Source Slot'])['GBIR (nm)'].std(),2).mean()
    GBIR = GBIR**.5
    
    center_tks = np.power(df.groupby(by=['Source Slot'])['Center Thickness (nm)'].std(),2).mean()
    center_tks = center_tks**.5
    
    Bow_BF = np.power(df.groupby(by=['Source Slot'])['GMLYMCD (Bow-BF) (nm)'].std(),2).mean()
    Bow_BF = Bow_BF**.5
    
    Warp_CF = np.power(df.groupby(by=['Source Slot'])['Warp-CF (nm)'].std(),2).mean()
    Warp_CF = Warp_CF**.5
    return [('Bow-BF',res,EE,Bow_BF),('Warp-CF',res,EE,Warp_CF),
            ('GBIR',res,EE,GBIR),('Center thickness',res,EE,center_tks)]  

In [179]:
def SFQx(df):
    """
    compute precision for SFQx file
    
    Arg:
    ====
    df: SFQx data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    if fullsite_only:
        df = df[df['SFQX Type']=='Full']
        
    SFQR = np.power(df.groupby(by=['Source Slot','Site Number'])['SFQR Value (nm)'].std(),2).mean()
    SFQR = SFQR**.5
    SFQX = np.power(df.groupby(by=['Source Slot','Site Number'])['SFQX Value (nm)'].std(),2).mean()
    SFQX = SFQX**.5
    return [('SFQR',res,EE,SFQR),('SFQX',res,EE,SFQX)]
    

In [180]:
def ESFQX(df):
    """
    compute precision for ESFQX file
    
    Arg:
    ====
    df: ESFQX data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    ESFQR = np.power(df.groupby(by=['Source Slot','ESFQR Angle (deg)'])['ESFQR Value (nm)'].std(),2).mean()
    ESFQR = ESFQR**.5
    ESFQX = np.power(df.groupby(by=['Source Slot','ESFQR Angle (deg)'])['ESFQX Value (nm)'].std(),2).mean()
    ESFQX = ESFQX**.5
    return [('ESFQR',res,EE,ESFQR),('ESFQX',res,EE,ESFQX)]

In [181]:
def frontNT(df):
    """
    compute precision for frontNT file
    
    Arg:
    ====
    df: frontNT data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    if fullsite_only:
        df = df[df['Front Site Based NT @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) PV Type']=='Full']
    frontNT_PV = np.power(df.groupby(by=['Source Slot','Site Number'])['Front Site Based NT @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) PV Value (nm)'].std(),2).mean()
    frontNT_PV = frontNT_PV**0.5
    return [('FrontNT PV',res,EE,frontNT_PV)]

In [182]:
def backNT(df):
    """
    compute precision for backNT file
    
    Arg:
    ====
    df: backNT data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    if fullsite_only:
        df = df[df['Back Site Based NT @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) PV Type']=='Full']
    backNT_PV = np.power(df.groupby(by=['Source Slot','Site Number'])['Back Site Based NT @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) PV Value (nm)'].std(),2).mean()
    backNT_PV = backNT_PV**0.5
    return [('BackNT PV',res,EE,backNT_PV)]  

In [183]:
def LSC(df):
    """
    compute precision for LSC file
    
    Arg:
    ====
    df: LSC data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    if fullsite_only:
        df = df[df['Shape Mean LSC @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) Type']=='Full']
    
    LSCmean_mean = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Mean LSC @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) Mean (nm / mm^2)'].std(),2).mean()
    LSCmean_mean = LSCmean_mean**0.5
    
    LSCmean_PV = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Mean LSC @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) PV (nm / mm^2)'].std(),2).mean()
    LSCmean_PV = LSCmean_PV**0.5
    return [('LSC-Mean mean',res,EE,LSCmean_mean),('LSC-Mean PV',res,EE,LSCmean_PV)]

In [184]:
def IPD_site(df):
    """
    compute precision for IPD site-level file
    
    Arg:
    ====
    df: IPD site-level data, dataframe
    
    Return:
    ====
    list of standard deviation for 4 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    if fullsite_only:
        df = df[df['Shape Residual X IPD @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) Type']=='Full']
    
    IPD_X_3sigma = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Residual X IPD @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) 3Sigma (nm / mm)'].std(),2).mean()
    IPD_X_3sigma = IPD_X_3sigma**0.5

    IPD_Y_3sigma = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Residual Y IPD @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) 3Sigma (nm / mm)'].std(),2).mean()
    IPD_Y_3sigma = IPD_Y_3sigma**0.5
    
    IPD_X_mean = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Residual X IPD @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) Mean (nm / mm)'].std(),2).mean()
    IPD_X_mean = IPD_X_mean**0.5
    
    IPD_Y_mean = np.power(df.groupby(by=['Source Slot','Site Number'])['Shape Residual Y IPD @ (26mmx33mm) & (0mm 0mm) & (0mmx0mm) Mean (nm / mm)'].std(),2).mean()
    IPD_Y_mean = IPD_Y_mean**0.5
    
    return [('IPD-X mean (nm/mm)',res,EE,IPD_X_mean),('IPD-Y mean (nm/mm)',res,EE,IPD_Y_mean),
            ('IPD-X 3sigma (nm/mm)',res,EE,IPD_X_3sigma),('IPD-Y 3sigma (nm/mm)',res,EE,IPD_Y_3sigma)]

In [185]:
def ZDD(df):
    """
    compute precision for ZDD file
    
    Arg:
    ====
    df: ZDD data, dataframe
    
    Return:
    ====
    list of standard deviation for 2 desired metrics
    """
    #ind, res, EE = header_info(filename)
    #df = pd.read_csv(filename,skiprows=ind)
    
    front_ZDD = np.power(df.groupby(by=['Source Slot','Sector Angle (deg)'])['Front Sector ZDD  Sectors: 16 @ 148 mm (nm / mm^2)'].std(),2).mean()
    front_ZDD = front_ZDD**.5
    
    back_ZDD = np.power(df.groupby(by=['Source Slot','Sector Angle (deg)'])['Back Sector ZDD  Sectors: 16 @ 148 mm (nm / mm^2)'].std(),2).mean()
    back_ZDD = back_ZDD**.5
    
    return [('Front ZDD @ 148 mm',res,EE,front_ZDD),('Back ZDD @ 148 mm',res,EE,back_ZDD)]

In [186]:
def precision_comp(df, file_path):
    """
    compute precision based on file type
    
    Args:
    ====
    df: input data, dataframe
    file_path: input data path, string
    
    Return:
    ====
    corresponding precision metrics, dataframe
    """
    
    if 'ESFQx_72Sector_2EE' in file_path: return ESFQX(df)
    if 'IMP_M49' in file_path: return M49(df)
    if 'siteNT_Back' in file_path: return backNT(df)
    if 'siteNT_Front' in file_path: return frontNT(df)
    if 'LSC' in file_path: return LSC(df)
    if 'IPD_Shape_10term_26x33' in file_path: return IPD_site(df)
    if 'SFQx_26x8' in file_path: return SFQx(df)
    if 'ZDD' in file_path: return ZDD(df)

In [187]:
#compute 1-day precision
if three_day == False:
    precision = []
    for file_path in glob.glob(input_file):
        ind, res, EE = header_info(file_path)
        df = pd.read_csv(file_path,skiprows=ind)
        if precision_comp(df, file_path):
            precision.extend(precision_comp(df,file_path))
    result = pd.DataFrame(precision, columns=['metric','resolution','EE','std_dev'])

1


In [188]:
#compute 3-day precision
if three_day == True:
    precision = []
    for f1 in glob.glob(input_file):
        fname1 = os.path.basename(f1)
        for f2 in glob.glob(input_file2):
            fname2 = os.path.basename(f2)
            for f3 in glob.glob(input_file3):
                fname3 = os.path.basename(f3)
                if fname1[:-17]==fname2[:-17] and fname1[:-17]==fname3[:-17]:
                    ind, res, EE = header_info(f1)
                    ls = [pd.read_csv(f1, skiprows=ind)]
                    ls.append(pd.read_csv(f2, skiprows=ind))
                    ls.append(pd.read_csv(f3, skiprows=ind))
                    df = pd.concat(ls, axis=0, ignore_index=True)
                    if precision_comp(df,f1):
                        precision.extend(precision_comp(df, f1))
                        break
                       
    result = pd.DataFrame(precision, columns=['metric','resolution','EE','std_dev'])

In [190]:
result.to_csv(output_path, index=None)